# Construção da base final - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando dados processados

Realizando as construções dos últimos dados finais do modelo.

### Componente

Os componentes finais do modelo são os mesmos do FooDB. Nesse caso, eles sofreram uma alteração em relação ao modelo originalmente proposto de que agora o ID deles será usado. Como foi discuto, com o desenvolvimento do projeto, percebeu-se que um número para os ids seria mais fácil do que fazer associações pelo nome, porque os nomes são muitos extensos. Esses dados já foram exportados no Notebook do FooDB.

### OrgaoPublico

Os dados de recomendações de nutrientes encontrados na base Dietary Reference Intakes (DRI) estão disponíveis online na forma de tabelas. Em relação à origem das recomendações, elas correspondem a apenas um órgão público, que é o NIH_US. Entretanto, separar essa informação das recomendações foi feito para manter os dados normalizados, pensando inclusive em possíveis integrações futuras com novos dados de recomendações.

Importando o único dado:

In [2]:
DROP TABLE IF EXISTS Orgao;

CREATE TABLE Orgao (
  Nome VARCHAR(10),
  PRIMARY KEY (Nome)
) AS SELECT
  Nome
FROM CSVREAD('../data/external/orgaopublico.csv');


Visualizando:

In [3]:
SELECT * FROM Orgao;


NIH_US

Exportando para os dados finais:

In [4]:
CALL CSVWRITE('../data/processed/database/orgaopublico.csv',
              'SELECT * FROM Orgao');


1

### IngredientesDasReceitas

Os ingredientes das receitas foram construídos nos notebooks referentes a cada base. Houve um cuidado em manter esses dados segundo o modelo proposto para que a construção final fosse mais fácil. Nesse caso, como todos os dados foram montados pensando no esquema final, basta apenas unir todos os dados em um único arquivo. Uma menção importante é que esse modelo também sofreu alterações. Devido a principalmente nomes de receitas muito extensos (alguns chegam a quase 300 caracteres), optou-se por utilizar o id das receitas como chave e não mais o nome.

Importando os dados de cada tabela

In [5]:
DROP TABLE IF EXISTS IDRFlavor;
DROP TABLE IF EXISTS IDRCulinary;
DROP TABLE IF EXISTS IDRRecipe;

CREATE TABLE IDRFlavor (
  Id INT,
  Banco VARCHAR(20),
  Ingrediente VARCHAR(80),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(20),
  PRIMARY KEY (Id, Banco, Ingrediente, Unidade)
) AS SELECT
  IDReceita, BANCO, INGREDIENTE, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/interim/flavor network/ingredientesdasreceitaspadronizados.csv');

CREATE TABLE IDRCulinary (
  Id INT,
  Banco VARCHAR(20),
  Ingrediente VARCHAR(80),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(20),
  PRIMARY KEY (Id, Banco, Ingrediente, Unidade)
) AS SELECT
  IDRECEITA, BANCOORIGINAL, FOODB, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/interim/culinarydb/receitasfinais.csv');

CREATE TABLE IDRRecipe (
  Id INT,
  Banco VARCHAR(20),
  Ingrediente VARCHAR(80),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(20),
  PRIMARY KEY (Id, Banco, Ingrediente, Unidade)
) AS SELECT
  IDRECEITA, 'RecipeNLG', FOODB, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/interim/recipenlg/quantidadesfinais.csv');


Visualizando:

In [6]:
SELECT * FROM IDRFlavor LIMIT 10;


In [7]:
SELECT * FROM IDRCulinary LIMIT 10;


In [8]:
SELECT * FROM IDRRecipe LIMIT 10;


Unindo todos os dados em uma única VIEW:

In [9]:
DROP VIEW IF EXISTS IngredientesDasReceitas;

CREATE VIEW IngredientesDasReceitas AS
    SELECT * FROM IDRFlavor
    UNION
    SELECT * FROM IDRCulinary
    UNION
    SELECT * FROM IDRRecipe
    ORDER BY Id;


Visualizando:

In [10]:
SELECT * FROM IngredientesDasReceitas LIMIT 10;


Exportando os dados

In [11]:
CALL CSVWRITE('../data/processed/database/ingredientesdasreceitas.csv',
              'SELECT * FROM IngredientesDasReceitas');


1820286

### ComponentesDosIngredientes

A associação entre componentes e ingredientes já foi feita no notebook de processamento dos dados do FooDB. Como foi explicado lá, os ingredientes novos criados não terão quantidades associadas porque o objetivo deles é servir apenas como base para integração de receitas. No entanto, optou-se por associar aos ingredientes compostos do CulinaryDB quantidades de componentes.

Como os ingredientes compostos não estão divididos por quantidades de cada um, supõe-se que cada um corresponde a uma parte igual do ingrediente composto final. Com dados mais precisos, essa etapa poderia ser melhorada.

Importando os componentes de cada ingrediente composto, bem como os componentes dos alimentos.

In [12]:
DROP TABLE IF EXISTS ComposicaoCompostos;
DROP TABLE IF EXISTS ComponentesDosIngredientes;

CREATE TABLE ComposicaoCompostos (
  Composto VARCHAR(80),
  Original VARCHAR(80),
  PRIMARY KEY (Composto, Original)
) AS SELECT
  INGREDIENTECOMPOSTO, INGREDIENTEORIGINAL
FROM CSVREAD('../data/processed/database/ingredientecomposto.csv');

CREATE TABLE ComponentesDosIngredientes (
  Ingrediente VARCHAR(80),
  Componente INT,
  Tipo VARCHAR(30),
  Quantidade DECIMAL(10, 3),
  Unidade VARCHAR(30),
  PRIMARY KEY (Ingrediente, Componente, Tipo, Unidade)
) AS SELECT
  NOME, IDCOMPONENTE, TIPOCOMPONENTE, QUANTIDADE, UNIDADE
FROM CSVREAD('../data/interim/foodb/componentesdosingredientes.csv');


Visualizando:

In [13]:
SELECT * FROM ComposicaoCompostos LIMIT 10;


In [14]:
SELECT * FROM ComponentesDosIngredientes LIMIT 10;


Criando uma VIEW nova com os dados dos componentes dos ingredientes compostos:

In [15]:
DROP VIEW IF EXISTS ComponentesCompostos;

CREATE VIEW ComponentesCompostos AS
    SELECT C.Composto, I.Componente, I.Tipo, AVG(I.Quantidade) Quantidade, I.Unidade
    FROM ComposicaoCompostos C, ComponentesDosIngredientes I
    WHERE C.Original = I.Ingrediente
    GROUP BY C.Composto, I.Componente, I.Tipo, I.Unidade
    ORDER BY C.Composto, I.Componente;


Visualizando:

In [16]:
SELECT * FROM ComponentesCompostos LIMIT 10;


Adicionando esses dados aos dados dos componentes:

In [17]:
INSERT INTO ComponentesDosIngredientes
SELECT * FROM ComponentesCompostos;


Exportando:

In [18]:
CALL CSVWRITE('../data/processed/database/componentesdosingredientes.csv',
              'SELECT * FROM ComponentesDosIngredientes');


99190

### Recomendacao

Os dados das recomendações foram extraídos segundo o modelo proposto das tabelas disponíveis no site da base. O trabalho foi feito manualmente e, por isso, a integração já foi garantida de imediato com os nomes dos componentes na base do FooDB. Para isso, foi feita uma busca pelo nome do componente na base e o ajuste já foi feito diretamente no processo de extração dos dados da web.

Resta, entretanto, associar esses dados pelo id + tipo, ao invés de pelo nome (como fora proposto no modelo inicial). Isso pode ser feito da seguinte forma:

In [19]:
DROP TABLE IF EXISTS Recomendacao;
DROP TABLE IF EXISTS Componente;

CREATE TABLE Recomendacao (
  Orgao VARCHAR(20),
  NomeComponente VARCHAR(100),
  FaixaEtaria VARCHAR(10),
  QuantidadeMg DECIMAL(10, 3),
  PRIMARY KEY (Orgao, NomeComponente, FaixaEtaria)
) AS SELECT
  Orgao, Componente, FaixaEtaria, QuantidadeMg
FROM CSVREAD('../data/external/recomendacao.csv');

CREATE TABLE Componente (
  ID INT,
  Nome VARCHAR(300),
  Tipo VARCHAR(20),
  PRIMARY KEY (ID, Tipo)
) AS SELECT
  ID, NOME, TIPO
FROM CSVREAD('../data/processed/database/componente.csv');


Visualizando:

In [20]:
SELECT * FROM Recomendacao LIMIT 10;


In [21]:
SELECT * FROM Componente LIMIT 10;


Criando uma VIEW com a associação por id e tipo:

In [22]:
DROP VIEW IF EXISTS RecomendacaoFinal;

CREATE VIEW RecomendacaoFinal AS
SELECT R.Orgao, C.ID, C.Tipo, R.FaixaEtaria, R.QuantidadeMg
FROM Recomendacao R, Componente C
WHERE R.NomeComponente = C.Nome;


Visualizando:

In [23]:
SELECT * FROM RecomendacaoFinal LIMIT 10;


Exportando:

In [24]:
CALL CSVWRITE('../data/processed/database/recomendacao.csv',
              'SELECT * FROM RecomendacaoFinal');


320